In [ ]:
import eikon as ek
import xml.etree.ElementTree as ET
from datetime import datetime
from tzlocal import get_localzone
import pytz
import numpy as np
import os

# Replace this with your Eikon app key created using App Key Generator app
eikon_app_key = os.environ['EIKON_APP_KEY']
ek.set_app_key(eikon_app_key)

def program():
    # Opening Eikon config files
    eikon_user_cache_dir = os.environ['LOCALAPPDATA'] + '/Thomson Reuters/Eikon User/Cache/'
    eikon_user_xml = ET.parse(eikon_user_cache_dir + 'LMO.LatestUserAccount.xml')
    eikon_user_uuid = eikon_user_xml.getroot().attrib['uuid']
    field_mapping = eikon_user_cache_dir + eikon_user_uuid + '/PersistentFiles/Config/Fieldmapping.xml'
    enumtype = eikon_user_cache_dir + eikon_user_uuid + '/PersistentFiles/Config/RealTime/Display Templates/enumtype.def'

    field_mapping_xml = ET.parse(field_mapping)
    
    d = {}
    # Creating a dictionary of exchange codes
    with open(enumtype) as f:
        start = 1000000
        for i, line in enumerate(f):
            if 'RDN_EXCHID     4' in line:
                start = i
            if i > start and line[0] != '!':
                key = line[16:19]
                val = line[4:7].strip()
                d[key] = val
            if 'PRCTCK_1      14' in line:
                break
                
    print('\n Hi ' + eikon_user_xml.getroot().attrib['userId'].split('.')[0].capitalize() + ', let me assist you with CF field mapping')
    
    # Input of RIC and cf_field. RIC exists if dataframe is not empty. 
    while True:
        ric = input('RIC (case sensitive): ')
        print('Searching...')
        df, err = ek.get_data(ric, ['RDN_EXCHID', 'PREF_DISP', 'RDNDISPLAY', 'RECORDTYPE'])
        if df.iloc[0].isnull().sum() == 4:
            print('RIC ' + ric + ' not found. The RIC does not exist or you are not entitled to view it')
            continue
        else:
            while True:
                CF_FIELDS = ['CF_LAST', 'CF_HIGH', 'CF_LOW', 'CF_BID', 'CF_ASK', 'CF_YIELD', 'CF_SOURCE', 'CF_LOTSIZE',
                             'CF_DATE', 'CF_TIME', 'CF_TICK', 'CF_NETCHNG', 'CF_EXCHNG', 'CF_VOLUME', 'CF_CLOSE', 'CF_NAME',
                             'CF_CURR']
                cf_field = input('CF_ field in format CF_XXXXX: ')
                if cf_field.upper() not in CF_FIELDS:
                    print('Incorrect CF_ field. Please choose from the list below:')
                    print(CF_FIELDS)
                    continue                
                
                else:
                    # Defining Exchange, PrefDisp, RDNDisplay, Recordtype, which determine logic of CF fields in fieldmapping.xml
                    df.fillna('Default', inplace=True)
                    if df.iloc[0,1] == 'Default':
                        rdn_exch = 'Default'
                    else:
                        rdn_exch = d[df.iloc[0, 1]]
                    pref_disp = df.iloc[0, 2]
                    rdn_display = df.iloc[0, 3]
                    record_type = df.iloc[0, 4]
                    
                    template_list = []
                    dictionary = {}
                    
                    # Iterating through fieldmapping.xml to find template/logic used for specific RIC/CF_FIELD
                    # First we create the list of templates that match the content of the RIC either on the value of each of the fields
                    # 'RDN_EXCHID', 'PREF_DISP', 'RDNDISPLAY', 'RECORDTYPE' or on the 'Default' value of the respective attribute in the template
                    for mapper in field_mapping_xml.getroot().iter('Mapper'):
                        if mapper.attrib['Field'] == cf_field.upper():
                            for field_map in mapper:
                                if ((field_map.attrib['RecordType'] == str(record_type) or field_map.attrib['RecordType'] == "Default") and 
                                    (field_map.attrib['Exchange'] == str(rdn_exch) or field_map.attrib['Exchange'] == 'Default') and 
                                    (field_map.attrib['PrefDisp'] == str(pref_disp) or field_map.attrib['PrefDisp'] == 'Default') and 
                                    (field_map.attrib['RDNDisplay'] == str(rdn_display) or field_map.attrib['RDNDisplay'] == 'Default')):
                                    template_list.append(field_map)
                            break
                    
                    # If there's more than one template the provides possible match for the RIC we select the one that provides the best match
                    # based on the most field values of the fields 'RDN_EXCHID', 'PREF_DISP', 'RDNDISPLAY', 'RECORDTYPE'
                    for template in template_list:
                        n = 0
                        if template.attrib['RecordType'] == 'Default':
                            n = n + 1
                        if template.attrib['PrefDisp'] == 'Default':
                            n = n + 1
                        if template.attrib['RDNDisplay'] == 'Default':
                            n = n + 1
                        if template.attrib['Exchange'] == 'Default':
                            n = n + 1
                        dictionary[n] = template
                    
                    template_used = dictionary[min(dictionary.keys())]

                    print(template_used.attrib)
                    print('\n Field ' + cf_field + ' is populated in the following priority order depending on available data under FID/Field:\n')
                    
                    # Gathering all data items to print current values of CF_ field and all data items values from the chosen template/logic
                    field_name_list = [cf_field]
                    for element in template_used:
                        print(element.attrib['FID'] + '\t ' + element.attrib['Field'] + '\n')
                        field_name = element.attrib['Field']
                        field_name_list.append(field_name)
                    # Empty values on CF values are possible and can't crash the script
                    df2, err = ek.get_data(ric,cf_field)
                    if df2.isnull().any().any():
                        print('\n' + cf_field + ' is currently empty\n')
                        break
                    
                    field_name_list.extend(['CF_DATE', 'CF_TIME'])
                    field_values_df, err = ek.get_data(ric, field_name_list)
                    
                    print('At the moment the above FIDs/Fields have the following values:\n')
                    print(field_values_df.iloc[:, :-2])
                    # Checking which FID currently provides value for the CF field.
                    # Replace cells containing only white spaces with NaN
                    field_values_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
                    field_values_df.dropna(axis=1, inplace=True)
                    if field_values_df.columns.values[2][:3] != "CF_":
                        print('\n')
                        print('Currently ' + cf_field + ' is populated by ' + field_values_df.columns.values[2] + '\n')
                    
                    # Eikon Data APIs return the values of CF_DATE and CF_TIME fields in GMT
                    # Here the values are converted to timezone of the operating system
# The logic here is flawed. 
# If CF_DATE or CF_TIME is empty it would have been removed from the dataframe when checking which FID populates the CF field
                    if cf_field in ['CF_DATE', 'CF_TIME']:
                        tz = get_localzone()
                        if 'CF_DATE' in field_values_df.columns and 'CF_TIME' in field_values_df.columns:
                            datestring = field_values_df.iloc[0, -2] + " " + field_values_df.iloc[0, -1]
                            try:
                                source_date = datetime.strptime(datestring, '%Y-%m-%d %H:%M:%S')
                                source_time_zone = pytz.timezone('GMT')
                                source_date_with_timezone = source_time_zone.localize(source_date)
                                target_date_with_timezone = source_date_with_timezone.astimezone(tz)
                                local_date = target_date_with_timezone.strftime('%Y-%m-%d %H:%M:%S')
                                print('CF_DATE and CF_TIME outputs are expressed in GMT timezone\n' +
                                      'The corresponding value of CF_DATE and CF_TIME in your local time is:\n' +
                                      local_date)
                            except:
                                print('Conversion to local timezone failed')
                        else:
                            print('Conversion to local timezone is not available because either CF_DATE or CF_TIME is empty')

                    break
            break

#Loop to allow checking multiple RICs without restarting the script.            
flag = True
while flag:
    program()
    print("\n")
    flag = input('Would you like to check another RIC? [Y/N]').upper() == 'Y'
    
print('The program will now terminate')
